In [25]:
import urllib.request
import re
from importlib.metadata import version
import tiktoken

## Downloading the text "The verdict"

In [2]:
url = ('https://raw.githubusercontent.com/rasbt/' 
       'LLMs-from-scratch/main/ch02/01_main-chapter-code/'
       'the-verdict.txt')
file_path = '../data/the-verdict.txt'
urllib.request.urlretrieve(url, file_path)

('../data/the-verdict.txt', <http.client.HTTPMessage at 0x769340285650>)

## Implementing a basic tokenizer

In [3]:
with open('../data/the-verdict.txt', 'r', encoding='utf-8') as f:
    raw_text = f.read()
print(f'Total number of character:: {len(raw_text)}')
print(raw_text[:99])

preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(f'Total number of tokens: {len(preprocessed)}')

Total number of character:: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 
Total number of tokens: 4690


## Assigning token IDs

In [4]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)
print(vocab_size)

1130


In [5]:
vocab = {token: integer for integer, token in enumerate(all_words)}
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


## Implementing a simple tokenizer class

In [15]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s, i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = ' '.join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [7]:
tokenizer = SimpleTokenizerV1(vocab)
text = """"It's the last he painted, you know,"
Mrs. Gisburn said with pardonable pride."""
print(text)
ids = tokenizer.encode(text)
print(ids)

"It's the last he painted, you know,"
Mrs. Gisburn said with pardonable pride.
[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [8]:
print(tokenizer.decode(ids))

" It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


## Adding special tokens

In [9]:
all_tokens = sorted(set(preprocessed))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])
vocab = {token: integer for integer, token in enumerate(all_tokens)}
print(len(vocab.items()))

1132


In [12]:
list(vocab.items())[-5:]

[('younger', 1127),
 ('your', 1128),
 ('yourself', 1129),
 ('<|endoftext|>', 1130),
 ('<|unk|>', 1131)]

In [14]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s, i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [item if item in self.str_to_int else "<|unk|>" for item in preprocessed]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = ' '.join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [20]:
text1 = 'Hello, do you like tea?'
text2 = 'In the sunlit terraces of the palace.'
text = ' <|endoftext|> '.join((text1, text2))
print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [22]:
tokenizer = SimpleTokenizerV2(vocab)
encoded = tokenizer.encode(text)
encoded

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]

In [23]:
tokenizer.decode(encoded)

'<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.'

## Byte pair encoding

In [24]:
!uv add tiktoken

Resolved 118 packages in 3.03s                                       
Prepared 2 packages in 2.13s                                             
Installed 2 packages in 9ms                                 
 + regex==2025.11.3
 + tiktoken==0.12.0


In [26]:
version("tiktoken")

'0.12.0'

In [27]:
tokenizer = tiktoken.get_encoding('gpt2')

In [31]:
text = "Akwirw ier"
integers = tokenizer.encode(text, allowed_special={'<|endoftext|>'})
integers

[33901, 86, 343, 86, 220, 959]

In [32]:
strings = tokenizer.decode(integers)
strings

'Akwirw ier'